In [7]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, precision_score
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Flatten
from deepface import DeepFace

In [8]:
def load_and_preprocess_images(image_paths):
    preprocessed_images = []
    for image_path in image_paths:
        img = DeepFace.detectFace(image_path, enforce_detection=False)
        preprocessed_images.append(img)
    return np.array(preprocessed_images)

In [9]:

# Load your dataset from the CSV file
# Assume the CSV file has columns: "image_path", "race", "gender"
df = pd.read_csv('merged_feret_original.csv')

# Split the dataset into training and testing sets
train_df, test_df = train_test_split(df, test_size=0.2, random_state=42)


# Preprocess training and testing images
X_train = load_and_preprocess_images(train_df['image_path'].values)
X_test = load_and_preprocess_images(test_df['filename'].values)

# Define the model architectures

# DeepFace with OpenCV
model_deepface_opencv = DeepFace.build_model('opencv')

# DeepFace with RetinaFace
model_deepface_retinaface = DeepFace.build_model('retinaface')

# BaseCNN model
model_base_cnn = Sequential([
    Flatten(input_shape=(224, 224, 3)),
    Dense(128, activation='relu'),
    Dense(1, activation='sigmoid')
])

# Compile the models
model_deepface_opencv.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])
model_deepface_retinaface.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])
model_base_cnn.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

# Train the models
model_deepface_opencv.fit(X_train, train_df['gender'].values, epochs=10, batch_size=32, verbose=1)
model_deepface_retinaface.fit(X_train, train_df['gender'].values, epochs=10, batch_size=32, verbose=1)
model_base_cnn.fit(X_train, train_df['gender'].values, epochs=10, batch_size=32, verbose=1)

# Evaluate the models on the test set
y_pred_deepface_opencv = model_deepface_opencv.predict(X_test)
y_pred_deepface_retinaface = model_deepface_retinaface.predict(X_test)
y_pred_base_cnn = model_base_cnn.predict(X_test)

# Convert probabilities to binary predictions
y_pred_deepface_opencv = (y_pred_deepface_opencv > 0.5).astype(int)
y_pred_deepface_retinaface = (y_pred_deepface_retinaface > 0.5).astype(int)
y_pred_base_cnn = (y_pred_base_cnn > 0.5).astype(int)

# Calculate accuracy and precision for the "Female" group
accuracy_deepface_opencv = accuracy_score(test_df['gender'].values, y_pred_deepface_opencv)
precision_deepface_opencv = precision_score(test_df['gender'].values, y_pred_deepface_opencv)

accuracy_deepface_retinaface = accuracy_score(test_df['gender'].values, y_pred_deepface_retinaface)
precision_deepface_retinaface = precision_score(test_df['gender'].values, y_pred_deepface_retinaface)

accuracy_base_cnn = accuracy_score(test_df['gender'].values, y_pred_base_cnn)
precision_base_cnn = precision_score(test_df['gender'].values, y_pred_base_cnn)

# Print results
print(f"DeepFace (OpenCV) - Accuracy: {accuracy_deepface_opencv}, Precision: {precision_deepface_opencv}")
print(f"DeepFace (RetinaFace) - Accuracy: {accuracy_deepface_retinaface}, Precision: {precision_deepface_retinaface}")
print(f"BaseCNN - Accuracy: {accuracy_base_cnn}, Precision: {precision_base_cnn}")


KeyError: 'image_path'